# 0 General functions and parameters

## 0.1 Hyperparameters

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

# Camera calibration with chessboard images

In [ ]:
def load_imgs(path):
    return [mpimg.imread(e) for e in glob.glob(path)]

def show_img_pairs(imgs1, imgs2, title1='', title2='', cmap1=None, cmap2=None):
    for img1, img2 in zip(imgs1, imgs2):
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
        f.tight_layout()
        
        ax1.imshow(img1, cmap=cmap1)
        ax1.set_title(title1, fontsize=30)
    
        ax2.imshow(img2, cmap=cmap2)
        ax2.set_title(title2, fontsize=30)
        
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

def get_mapping(chessboard_imgs, chessboard_size):
    obj_pts = np.zeros((chessboard_size[0]*chessboard_size[1],3), np.float32)
    obj_pts[:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1,2)

    objs_pts = []
    imgs_pts = []

    for idx, img in enumerate(chessboard_imgs):
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

        if ret == True:
            objs_pts.append(obj_pts)
            imgs_pts.append(corners)
            cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
            
    return objs_pts, imgs_pts

chessboard_imgs = load_imgs('camera_cal/calibration*.jpg')
chessboard_shape = chessboard_imgs[0].shape[:2]
chessboard_size = (9, 6)

objs_pts, imgs_pts =  get_mapping(chessboard_imgs, chessboard_size)
_, mtx, dist, _, _ = cv2.calibrateCamera(objs_pts, imgs_pts, chessboard_shape, None, None)
    
def undistort(img):
    return cv2.undistort(img, mtx, dist, None, mtx)

undist_imgs = [undistort(img) for img in chessboard_imgs]
show_img_pairs(chessboard_imgs, undist_imgs, title1='original', title2='undistroted')

# Undistorting test images

In [ ]:
# HYPERPARAMETERS for image thresholding
# Color channel
s_thresh_min = 170
s_thresh_max = 255

# Sobel x
sobx_thresh_min = 20
sobx_thresh_max = 100
sobx_kernel_size = 3

# Sobel y
soby_thresh_min = 20
soby_thresh_max = 100
soby_kernel_size = 3

# Magnitude gradient
mag_thresh_min = 20
mag_thresh_max = 100
mag_kernel_size = 3

# Direction gradient
dir_thresh_min = 0.4
dir_thresh_max = 0.8
dir_kernel_size = 15

## 0.2 Plotting functions

In [ ]:
def plot_images_from_dict(input_dictionary, images_per_line = 5, figw = 15, figh = 15, save = False):
    """
    Plots all images of a dictionary into the jupyter notebook
    
    Input:
    input_dictonary (dict): Input a dictionary with file path as keys and images as values
    images_per_line (int): Defines how many images will be displayed per line
    figw (int): Defines the width of the overall output figure
    figh (int): Defines the hight of the overall output figure
    save (bool): Determines whether the images will be saved under the same image path with the filename extension 
                "_annotated" (useful if images were modified)    
    """
    
    # Define required number of lines and columns in plot to create subplots
    num_images = len(input_dictionary)
    images_per_column = int(math.ceil(num_images/images_per_line))
    
    # Create subplots
    fig, axes = plt.subplots(images_per_column,images_per_line,figsize = (figw,figh))
    
    # Remove axis for all subplots
    for i, ax in enumerate(axes.flat):
        ax.axis("off")
    
    # Display all images
    for ax, image in zip(axes.flat,sorted(input_dictionary.keys())):
        ax.imshow(input_dictionary[image])
        ax.set_title(image)
        # Save all images if "save"-function was activated (to be used if images were modified before)
        if save:
            img_out_name = "{}_annotated.png".format(image[:image.find(".")])
            plt.imsave(img_out_name,input_dictionary[image].astype(np.uint8))
        
    # Output plot with all images    
    plt.tight_layout()
    plt.axis("off")
    plt.show()

In [ ]:
def plot_image_comparison(img_before, img_after, img_name, annotation,cmap_before=None,cmap_after=None):
    """
    Plots a comparison between two images in the jupyter notebook
    
    Input:
    img_before (np.array): Input the initial image before the conversion is applied
    img_after (np.array): Input image after the conversion is applied
    img_name (string): Name of the image which should be displayed above the image description
    annotation (str): Input annotation to the image (conversion method)
    """
    images_per_column = 1
    images_per_line = 2
    figw = 13
    figh = 7
    fig, axes = plt.subplots(images_per_column,images_per_line,figsize = (figw,figh))
    for i, ax in enumerate(axes.flat):
        if i == 0:
            ax.imshow(img_before,cmap = cmap_before)
            ax.axis("off")
            ax.set_title("{}\nImage before {}".format(img_name, annotation))
        if i == 1:
            ax.imshow(img_after, cmap = cmap_after)
            ax.axis("off")
            ax.set_title("{}\nImage after {}".format(img_name, annotation))
    plt.tight_layout()
    plt.axis("off")
    plt.show()
    
import string
    
def save_image_incl_extension(img_after, initial_image_path, img_annotation):
    """
    Saves image after conversion to file
    
    Input:
    img_after (np.array): Image after conversion
    initial_image_path (str): Initial path from where the image is sourced
    img_annotation (str): Annotation to be added at the end of the initial image path
    """
    img_out_name = "{}_{}.png".format(initial_image_path[:initial_image_path.find(".")],img_annotation.replace(" ","_"))
    plt.imsave(img_out_name,img_after.astype(np.uint8))  

# 1 Camera calibration

## 1.1 Load images and fit corners

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
%matplotlib inline

# Define number of chessboard inside corner points
chess_corner_x = 9
chess_corner_y = 6

# Read in and create a list of calibration images
calibration_images = glob.glob("camera_cal/calibration*.jpg")

# Add container for filenames of images with detected edges and non-detected edges
img_corner_det_true = {}
img_corner_det_false = {}

# Array containers to store object and image points from all images
obj_points = [] # 3D points in real world
img_points = [] # 2D points in image

# Prepare object points like (0,0,0), (1,0,0), (2,0,0), ...., (7,5,0)
objp = np.zeros((chess_corner_x * chess_corner_y,3), np.float32)
objp[:,:2] = np.mgrid[0:chess_corner_x,0:chess_corner_y].T.reshape(-1,2) # x, y coordinates  

for fname in calibration_images:
    # Read in each image
    image = mpimg.imread(fname)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray,(chess_corner_x,chess_corner_y), None)
    
    # If corner points are found, add object points, image points
    if ret == True:
        img_points.append(corners)
        obj_points.append(objp)
        
        # Draw and display the corners
        image = cv2.drawChessboardCorners(image,(chess_corner_x, chess_corner_y), corners, ret)
        
        # Append images with corners to dictionary of images
        img_corner_det_true[fname] = image        
    
    else:
        # Append images with no corners identified to dictionary of images
        img_corner_det_false[fname] = image   

In [ ]:
# Return plot of all images with corners found (and save them to "..._annotated.png"-files) 
# -> To execute second part in parentheses set last function argument to "True" 
# (not activated as computation takes a few seconds) 
print("\nAll images with identified corners:")
plot_images_from_dict(img_corner_det_true,3,15,20,False)

# Return plot of all images for which no corners have been identified (verify whether reason is that not all required corners are on the image)
print("\nAll images for which no corners could be identified:")
plot_images_from_dict(img_corner_det_false,3,15,6,False)

## 1.2 Calibrate camera

In [ ]:
test_imgs = load_imgs('test_images/*.jpg')
undist_imgs = [undistort(img) for img in test_imgs]
show_img_pairs(test_imgs, undist_imgs, title1='original', title2='undistorted')

# Filters for thresholding

In [ ]:
def sobel_thresh(img, orient='x', ksize=3, thresh=(20, 100)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    dx = 1 if orient == 'x' else 0
    dy = 1 if orient == 'y' else 0
    sobel = cv2.Sobel(gray, cv2.CV_64F, dx, dy, ksize=ksize)
    abs_sobel = np.absolute(sobel)
    scaled = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    mask = np.zeros_like(scaled)
    mask[(thresh[0] <= scaled) & (scaled <= thresh[1])] = 1
    return mask

def mag_thresh(img, ksize=5, thresh=(40, 100)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize)
    mag = np.sqrt(np.square(sobelx) + np.square(sobely))
    scaled = np.uint8(255 * mag / np.max(mag))
    mask = np.zeros_like(scaled)
    mask[(thresh[0] <= scaled) & (scaled <= thresh[1])] = 1
    return mask

def direct_thresh(rgb, ksize=5, thresh=(0.7, 1.3)):
    gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize)
    absx = np.abs(sobelx)
    absy = np.abs(sobely)
    direct = np.arctan2(absy, absx)
    mask = np.zeros_like(direct)
    mask[(thresh[0] <= direct) & (direct <= thresh[1])] = 1
    return mask

def hls_thresh(img, channel=2, thresh=(160, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_ch = hls[:,:,channel]
    mask = np.zeros_like(s_ch)
    mask[(s_ch >= thresh[0]) & (s_ch <= thresh[1])] = 1
    return mask

def comb_thresh(img):
    sobelx = sobel_thresh(img, 'x', ksize=9, thresh=(20, 100))
    s = hls_thresh(img, channel=2, thresh=(160, 255))
    direct = direct_thresh(img, ksize=15, thresh=(0.7, 1.3))
    combined = np.zeros_like(sobelx)
    combined[((sobelx == 1) | (s == 1)) & (direct == 1)] = 1
    return combined

binary_imgs = [comb_thresh(e) for e in undist_imgs]
show_img_pairs(undist_imgs, binary_imgs, cmap2='gray', title1='undistorted', title2='binary')

# Perspective transform

In [ ]:
straight_img = undist_imgs[4]

src_pts = np.float32([
    [275, 670],
    [1028, 670],
    [596, 450],
    [685, 450]
])

height, width = straight_img.shape[:2]
offset = 250

dst_pts = np.float32([
    [offset, height - 5],
    [width - offset, height - 5],
    [offset, 5],
    [width - offset, 5]
])

M = cv2.getPerspectiveTransform(src_pts, dst_pts)

def warp(img):
    return cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)

M_inv = np.linalg.inv(M)

def unwarp(img):
    height, width = img.shape[:2]
    return cv2.warpPerspective(img, M_inv, (width, height))

warped_img = warp(straight_img)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
fig.tight_layout()

ax1.imshow(straight_img)
ax1.plot(src_pts[:,0], src_pts[:, 1], 'ro', markersize=10)
ax1.set_title('source points', fontsize=30)   

ax2.imshow(warped_img)
ax2.plot(dst_pts[:,0], dst_pts[:, 1], 'ro', markersize=10)
ax2.set_title('destination points', fontsize=30)
plt.show()

In [ ]:
warped_imgs = [warp(img) for img in binary_imgs]
show_img_pairs(binary_imgs, warped_imgs, cmap1='gray', cmap2='gray', title1='binary', title2='transformed')

# Pipeline for processing test images

In [ ]:
def find_next_x(prev_x, offset, margin, convolved_layer):
    min_index = max(prev_x + offset - margin, 0)
    max_index = min(prev_x + offset + margin, width)
    convolved = convolved_layer[min_index:max_index]
    
    if sum(convolved) == 0:
        return None
    else:
        return np.argmax(convolved) + min_index - offset

def get_level_y(level, window_height, height):
    return height - (level * window_height + int(window_height / 2))

def find_points(binary_img, window_size, window_margin):
    left_pts = []
    right_pts = []
    
    heigth, width = binary_img.shape
    window_width, window_height = window_size
    kernel = np.ones(window_width)
    
    y = get_level_y(0, window_height, height)
    
    left_sum = np.sum(binary_img[int(3 / 4 * height):, :int(width / 2)], axis=0)
    left_x = np.argmax(np.convolve(kernel, left_sum)) - int(window_width / 2)
    left_pts.append((left_x, y))
    
    right_sum = np.sum(binary_img[int(3 / 4 * height):, int(width / 2):], axis=0)
    right_x = np.argmax(np.convolve(kernel, right_sum)) - int(window_width / 2) + int(width / 2)
    right_pts.append((right_x, y))
    
    num_levels = (int)(height / window_height)
        
    for level in range(1, num_levels):
        y = get_level_y(level, window_height, height)
        layer = np.sum(binary_img[height - (level + 1) * window_height:height - level * window_height,:], axis=0)
        convolved = np.convolve(kernel, layer)
        offset = int(window_width / 2)
    
        next_left_x = find_next_x(left_x, offset, window_margin, convolved)
        
        if next_left_x is not None:
            left_x = next_left_x
            left_pts.append((left_x, y))
        
        next_right_x = find_next_x(right_x, offset, window_margin, convolved)
        
        if next_right_x is not None:
            right_x = next_right_x
            right_pts.append((right_x, y))
    
    return (np.array(left_pts), np.array(right_pts))

def draw_window(img_size, pt, window_size):
    height, width = img_size
    window_width, window_height = window_size
    
    x, y = pt

    x_min = max(int(x-window_width/2), 0)
    x_max = min(int(x+window_width/2), width)
    
    y_min = max(int(y-window_height/2), 0)
    y_max = min(int(y+window_height/2), height)

    output_img = np.zeros(img_size)
    output_img[y_min:y_max, x_min:x_max] = 1
    return output_img

def fit_lane(points):
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

def draw_windows(binary_img, left_pts, right_pts, window_size):
    img_size = binary_img.shape
    overlay_img = np.zeros(img_size)
    
    for pt in left_pts:
        window_img = draw_window(img_size, pt, window_size)
        overlay_img[(window_img == 1) | (overlay_img == 1)] = 1
    
    for pt in right_pts:
        window_img = draw_window(img_size, pt, window_size)
        overlay_img[(window_img == 1) | (overlay_img == 1)] = 1
    
    overlay_img = np.array(overlay_img, np.uint8) * 255
    empty_img = np.zeros_like(overlay_img)
    overlay_img = np.array(cv2.merge((empty_img, overlay_img, empty_img)), np.uint8)
    color_img = np.dstack((binary_img, binary_img, binary_img))*128
    return cv2.addWeighted(color_img, 1, overlay_img, 0.5, 0.0)
    
def draw_fit_overlay(binary, left_points, right_points):
    (height, width) = binary.shape
    left_fit = np.polyfit(left_points[:,1], left_points[:,0], 2)
    right_fit = np.polyfit(right_points[:,1], right_points[:,0], 2)
    
    plot_y = np.linspace(0, height - 1, height)
    left_fit_x = left_fit[0]*plot_y**2+left_fit[1]*plot_y+left_fit[2]
    right_fit_x = right_fit[0]*plot_y**2+right_fit[1]*plot_y+right_fit[2]
    
    overlay = np.zeros_like(binary).astype(np.uint8)
    overlay = np.dstack((overlay, overlay, overlay))

    left_plot_points = np.array([np.transpose(np.vstack([left_fit_x, plot_y]))])
    right_plot_points = np.array([np.flipud(np.transpose(np.vstack([right_fit_x, plot_y])))])
    plot_points = np.hstack((left_plot_points, right_plot_points))

    cv2.fillPoly(overlay, np.int_([plot_points]), (0, 255, 0))
    return overlay

def draw_fit(binary, left_points, right_points):
    overlay = draw_fit_overlay(binary, left_points, right_points)
    color = np.dstack((binary, binary, binary))*255
    return cv2.addWeighted(color, 1, overlay, 0.5, 0.0)

def get_curvature(points, m_per_pix):
    x = points[:,0]*m_per_pix[0]
    y = points[:,1]*m_per_pix[1]
    y_eval = np.max(y)
    fit = np.polyfit(y, x, 2)
    return ((1+(2*fit[0]*y_eval+fit[1])**2)**1.5)/np.absolute(2*fit[0])

win_size = (50, 80)
win_margin = 100
m_per_pix= (3.7/700, 30.0/720)

window_imgs = []
fit_imgs = []
poly_imgs = []
unwarped_imgs = []
final_imgs = []

(height, width) = test_imgs[0].shape[:-1]
img_size = (width, height)    

for i in range(len(test_imgs)):
    binary = warped_imgs[i]
    left_pts, right_pts = find_points(binary, win_size, win_margin)
    left_curvature = get_curvature(left_pts, m_per_pix)
    right_curvature = get_curvature(right_pts, m_per_pix)
    
    window_imgs.append(draw_windows(binary, left_pts, right_pts, win_size))
    fit_imgs.append(draw_fit(binary, left_pts, right_pts))
    
    overlay_img = draw_fit_overlay(binary, left_pts, right_pts)
    poly_imgs.append(cv2.addWeighted(np.dstack((binary * 128, binary * 128, binary * 128)), 1, overlay_img, 0.5, 0.0))
    
    unwarped_img = unwarp(overlay_img)
    unwarped_imgs.append(unwarped_img)
    
    final_img = cv2.addWeighted(test_imgs[i], 1, unwarped_img, 0.5, 0.0)
    final_imgs.append(final_img)

## Windows

In [ ]:
show_img_pairs(warped_imgs, window_imgs, cmap1='gray', title1='transformed', title2='windows')

## Lane area

In [ ]:
test_images_s_thresh = {}
annotation = "S-binary-threshold"
for key in sorted(test_images_s_channel.keys()):
    test_images_s_thresh[key] = S_to_thresh(test_images_s_channel[key],s_thresh_min,s_thresh_max)
    plot_image_comparison(test_images_s_channel[key],test_images_s_thresh[key],key, annotation,cmap_before="gray",
                          cmap_after="gray")
plt.imsave("output_images/06_s_binary.png",test_images_s_thresh["test_images/test2.jpg"],cmap = "gray")

### 2.2.2 Create Sobel based binary image

In [ ]:
# Define sobel threshold functions
def abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    if orient == "x":
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel))
    if orient == "y":
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Apply threshold
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return grad_binary

In [ ]:
test_images_abssob_x = {}
annotation = "Sobelx"
for key in sorted(test_images_dst.keys()):
    test_images_abssob_x[key] = abs_sobel_thresh(test_images_dst[key], orient='x', sobel_kernel= sobx_kernel_size, 
                                                 thresh=(sobx_thresh_min, sobx_thresh_max))    
    plot_image_comparison(test_images_dst[key],test_images_abssob_x[key],key, annotation,cmap_before=None,
                          cmap_after="gray")
plt.imsave("output_images/07_abssob_x.png",test_images_abssob_x["test_images/test2.jpg"],cmap = "gray")

In [ ]:
test_images_abssob_y = {}
annotation = "Sobely"
for key in sorted(test_images_dst.keys()):
    test_images_abssob_y[key] = abs_sobel_thresh(test_images_dst[key], orient='y', sobel_kernel= soby_kernel_size, 
                                                 thresh=(soby_thresh_min, soby_thresh_max))    
    plot_image_comparison(test_images_dst[key],test_images_abssob_y[key],key, annotation,cmap_before=None,
                          cmap_after="gray")
plt.imsave("output_images/08_abssob_y.png",test_images_abssob_y["test_images/test2.jpg"],cmap = "gray")

In [ ]:
def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F,1,0, ksize = sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F,0,1, ksize = sobel_kernel)
    gradmag = np.sqrt(sobelx**2,sobely**2)
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    # Apply threshold
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0])&(gradmag <= mag_thresh[1])] = 1    
    return mag_binary

In [ ]:
test_images_mag_grad = {}
annotation = "magnitude gradient"
for key in sorted(test_images_dst.keys()):
    test_images_mag_grad[key] = mag_thresh(test_images_dst[key], sobel_kernel= mag_kernel_size, 
                                           mag_thresh=(mag_thresh_min, mag_thresh_max))    
    plot_image_comparison(test_images_dst[key],test_images_mag_grad[key],key, annotation,cmap_before=None,
                          cmap_after="gray")
plt.imsave("output_images/09_mag_grad.png",test_images_mag_grad["test_images/test2.jpg"],cmap = "gray")

In [ ]:
def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F,1,0, ksize = sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F,0,1, ksize = sobel_kernel)
    absgraddir = np.arctan2(np.absolute(sobelx),np.absolute(sobely))
    # Apply threshold
    dir_binary = np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return dir_binary

In [ ]:
test_images_dir_grad = {}
annotation = "direction gradient"
for key in sorted(test_images_dst.keys()):
    test_images_dir_grad[key] = dir_threshold(test_images_dst[key], sobel_kernel=dir_kernel_size, 
                                              thresh=(dir_thresh_min, dir_thresh_max))    
    plot_image_comparison(test_images_dst[key],test_images_dir_grad[key],key, annotation,cmap_before=None,
                          cmap_after="gray")
plt.imsave("output_images/10_dir_grad.png",test_images_dir_grad["test_images/test2.jpg"],cmap = "gray")

In [ ]:
def sob_to_sobcomb(img_gradx, img_grady, img_mag_grad, img_dir_grad):
    assert img_gradx.shape == img_grady.shape == img_mag_grad.shape == img_dir_grad.shape, "not all input images have the same shape"
    combined = np.zeros_like(img_gradx)
    combined[((img_gradx == 1) & (img_grady == 1)) | ((img_mag_grad == 1) & (img_dir_grad == 1))] = 1
    return combined  

In [ ]:
test_images_sobcomb = {}
annotation = "combined sobel"
for key in sorted(test_images_dst.keys()):
    test_images_sobcomb[key] = sob_to_sobcomb(test_images_abssob_x[key],test_images_abssob_y[key],
                                              test_images_mag_grad[key],test_images_dir_grad[key])
    plot_image_comparison(test_images_dst[key],test_images_sobcomb[key],key, annotation,cmap_before=None,
                          cmap_after="gray")
plt.imsave("output_images/11_sobcomb.png",test_images_sobcomb["test_images/test2.jpg"],cmap = "gray")

### 2.2.3 Create overall combined binary image

In [ ]:
show_img_pairs(warped_imgs, poly_imgs, cmap1='gray', title1='transformed', title2='lane polygon')

# Video pipeline

In [ ]:
def get_curvature(fit, y):
    return ((1+(2*fit[0]*y+fit[1])**2)**1.5)/np.absolute(2*fit[0])

def eval_x(fit, y):
    return fit[0]*y**2+fit[1]*y+fit[2]

def draw_lanes_poly(shape, left_fit, right_fit):
    (height, width) = shape
    
    y = np.linspace(0, height - 1, height)
    left_x = eval_x(left_fit, y)
    right_x = eval_x(right_fit, y)
    
    output = np.zeros((height, width, 3)).astype(np.uint8)

    left_points = np.array([np.transpose(np.vstack([left_x, y]))])
    right_points = np.array([np.flipud(np.transpose(np.vstack([right_x, y])))])
    plot_points = np.hstack((left_points, right_points))

    cv2.fillPoly(output, np.int_([plot_points]), (0, 255, 0))
    return output

def draw_text(img, text, position):
    font = cv2.FONT_HERSHEY_PLAIN
    font_scale = 2
    font_color = (0, 255, 0)
    line_type = 2
    cv2.putText(img, text, position, font, font_scale, font_color, line_type)

def find_close_points(binary_img, margin, left_fit, right_fit):
    nonzero = binary_img.nonzero()
    nonzero_x = np.array(nonzero[1])
    nonzero_y = np.array(nonzero[0])
    
    left_eval_x = eval_x(left_fit, nonzero_y)
    left_lane_inds = ((left_eval_x - margin) < nonzero_x) & (nonzero_x < (left_eval_x + margin))

    right_eval_x = eval_x(right_fit, nonzero_y)
    right_lane_inds = ((right_eval_x - margin) < nonzero_x) & (nonzero_x < (right_eval_x + margin))

    left_pts = np.stack([nonzero_x[left_lane_inds], nonzero_y[left_lane_inds]], axis=1)
    right_pts = np.stack([nonzero_x[right_lane_inds], nonzero_y[right_lane_inds]], axis=1)
    return left_pts, right_pts

class Lane():
    def __init__(self, img_size, m_per_px, last_n):
        self.last_n = last_n
        self.m_per_px = m_per_px
        self.img_size = img_size 
        
        self.fit = None        
        self.curvature = None
        self.offset = None
        self.slope = None

        self.last_fits = []

    def update(self, pts):
        if len(pts) < 2:
            self.fit = None
            return

        self.fit = np.polyfit(pts[:,1], pts[:,0], 2)
        
        pts_m = pts*self.m_per_px
        fit_m = np.polyfit(pts_m[:,1], pts_m[:,0], 2)
        max_y = self.img_size[1]
        max_y_m = max_y*self.m_per_px[1]
        self.curvature = get_curvature(fit_m, max_y_m)
        
        mid_x = self.img_size[0] / 2
        eval_max_x = eval_x(self.fit, max_y)
        self.offset = abs(eval_max_x - mid_x)*self.m_per_px[0]
        
        eval_min_x = eval_x(self.fit, 0)
        self.slope = (eval_max_x - eval_min_x) / max_y

    def accept(self):
        self.last_fits.insert(0, self.fit)
        self.last_fits = self.last_fits[:self.last_n]
        
    def reject(self):
        self.fit = None
            
    def smooth_fit(self):
        if len(self.last_fits):
            return np.mean(self.last_fits, axis=0)

class LaneFinder():
    def __init__(self, img_size, win_size, win_margin, m_per_px, n_last):
        self.img_size = img_size
        self.win_size = win_size
        self.win_margin = win_margin
        self.m_per_px = m_per_px
        self.n_last = n_last
        
        self.left_lane = Lane(img_size, m_per_px, n_last)
        self.right_lane = Lane(img_size, m_per_px, n_last)
        

    def process(self, img):
        output_img = np.copy(img)
        
        height, width = img.shape[:2]
        undist_img = undistort(img)
        binary_img = comb_thresh(undist_img)
        warped_img = warp(binary_img)
        
        if self.left_lane.fit is None or self.right_lane.fit is None:
            left_pts, right_pts = find_points(warped_img, self.win_size, self.win_margin)
        else: 
            left_pts, right_pts = find_close_points(warped_img, self.win_margin, self.left_lane.fit, self.right_lane.fit)
        
        self.left_lane.update(left_pts)
        self.right_lane.update(right_pts)
        
        if self.left_lane.fit is not None and self.right_lane.fit is not None:
            curvature_diff = abs(self.left_lane.curvature - self.right_lane.curvature)
            curvature_check = self.left_lane.curvature > 800 or self.right_lane.curvature > 800 or curvature_diff < 500
            
            separation = self.left_lane.offset + self.right_lane.offset
            separation_check = 2.6 < separation < 6.4
            
            slope_diff = abs(self.left_lane.slope - self.right_lane.slope)
            slope_check = slope_diff < 0.2
            
            offset = abs(self.left_lane.offset - self.right_lane.offset)
            
            curvature_text = 'Curvature left {:.0f}, right {:.0f}'.format(
                self.left_lane.curvature, self.right_lane.curvature)
            draw_text(output_img, curvature_text, (50, 50))
            
            offset_text = 'Offset left {:.3f}, right {:.3f}, car {:.3f}'.format(
                self.left_lane.offset, self.right_lane.offset, offset)
            draw_text(output_img, offset_text, (50, 100))
            
            slope_text = 'Slope left {:.3f}, right {:.3f}'.format(
                self.left_lane.slope, self.right_lane.slope)
            draw_text(output_img, slope_text, (50, 150))

            diff_text = 'Diff curvature {:.3f}, separation {:.3f}, slope {:.3f}'.format(
                curvature_diff, separation, slope_diff)
            draw_text(output_img, diff_text, (50, 200))
            
            check_text = 'Check curvature {}, separation {}, slope {}'.format(curvature_check, separation_check, slope_check)
            draw_text(output_img, check_text, (50, 250))
        
            if curvature_check and separation_check and slope_check:
                self.left_lane.accept()
                self.right_lane.accept()
            else:
                self.left_lane.reject()
                self.right_lane.reject()

        left_fit = self.left_lane.smooth_fit()
        right_fit = self.right_lane.smooth_fit()
        
        if left_fit is not None and right_fit is not None:
            poly_img = draw_lanes_poly(warped_img.shape, left_fit, right_fit)
            unwarped_img = unwarp(poly_img)
            output_img = cv2.addWeighted(output_img, 1, unwarped_img, 0.5, 0.0)
        
        return output_img

## Video pipeline can be also used to process images

In [ ]:
def process_image(img):
    lane_finder = LaneFinder(img_size, win_size, win_margin, m_per_pix, 1)
    return lane_finder.process(img)

proc_imgs = []

for img in test_imgs:
    proc_imgs.append(process_image(img))
    
show_img_pairs(test_imgs, proc_imgs)

## Utility functions for video processing

In [ ]:
def process_video(input_path, output_path):
    lane_finder = LaneFinder(img_size, win_size, win_margin, m_per_pix, 5)
    test_video = VideoFileClip(input_path)
    output_video = test_video.fl_image(lane_finder.process)
    %time output_video.write_videofile(output_path, audio=False)
    
def embed_video(path):
    return HTML("""
        <video width="960" height="540" controls>
          <source src="{0}">
        </video>
    """.format(path))

## Processing of test videos

In [ ]:
# Calibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points,img_points, gray.shape[::-1],None,None)

## 1.3 Perform distortion correction

In [ ]:
# Function for distortion correction on single image
def distortion_correction(img,mtx,dist):
    return cv2.undistort(img,mtx,dist,None,mtx)

In [ ]:
# Fit an undistorted test image of the chessboard
test_distorted = img_corner_det_false["camera_cal/calibration01.jpg"]
test_undistorted = distortion_correction(test_distorted,mtx,dist)
plot_image_comparison(test_distorted, test_undistorted, "camera_cal/calibration01.jpg","distortion correction",cmap_before=None,cmap_after=None)
plt.imsave("output_images/01_chess_dist.png",test_distorted)
plt.imsave("output_images/02_chess_dist.png",test_undistorted)

# 2 Pipeline (test images)

## 2.0 Load images

In [ ]:
import glob

img_paths = glob.glob("test_images/*.jpg")

In [ ]:
#img_path = "test_images/test1.jpg"
annotation = "distortion correction"
test_images = {}
for i, img_path in enumerate(img_paths):
    test_images[img_path] = mpimg.imread(img_path)
plt.imsave("output_images/03_original.png",test_images["test_images/test2.jpg"])

## 2.1 Distortion correction

In [ ]:
plt.imshow(test_images_dst["test_images/straight_lines1.jpg"])
plt.axis("off")
plt.plot(255,688,".")
plt.plot(1051,688,".")
plt.plot(595,452,".")
plt.plot(686,452,".")
plt.savefig("output_images/13_src_points")

In [ ]:
# Four source coordinates
src = np.float32(
    [[255,688],
     [1051,688],
     [595,452],
     [686,452]])

# Four desired coordinates
dst = np.float32(
    [[360,720],
     [946,720],
     [360,0],
     [946,0]])

In [ ]:
def warp(img,src,dst):
    
    # Define calibration box in source (origingal) and destination (desired or warped) coordinates
    img_size = (img.shape[1],img.shape[0])    
    
    # Compute the perspective transform, M
    M = cv2.getPerspectiveTransform(src, dst)
    
    # Create warped image - uses linear interpolation
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped   

In [ ]:
# Ensure that example image is warped correctly
example_warped = warp(test_images_dst["test_images/straight_lines1.jpg"],src,dst)
plt.imshow(example_warped)
plt.axis("off")
plt.plot(360,720,".")
plt.plot(944,720,".")
plt.plot(360,0,".")
plt.plot(946,0,".")
plt.savefig("output_images/14_dst_points")

In [ ]:
# Warp test images
test_images_warped = {}
for key in sorted(test_images_dst.keys()):
    test_images_warped[key] = warp(test_images_dst[key],src,dst)    
    plot_image_comparison(test_images_dst[key],test_images_warped[key],key, annotation,cmap_before=None,cmap_after=None)
plt.imsave("output_images/15_warped.png",test_images_warped["test_images/test2.jpg"])

In [ ]:
# Warp combined binary
test_images_warped_cb = {}
for key in sorted(test_images_dst.keys()):
    test_images_warped_cb[key] = warp(combined_binary[key],src,dst)    
    plot_image_comparison(test_images_warped[key],test_images_warped_cb[key],key, annotation,cmap_before=None,cmap_after="gray")
plt.imsave("output_images/16_warped_binary.png", test_images_warped_cb["test_images/test2.jpg"],cmap = "gray")

## 2.4 Identify lane-line pixels and fit with polynomial

In [ ]:
# Search for lanes if no line fitted so far

def identify_lane_line_first(img):
    """
    Input:
    img (np.array): The warped input image
    Output:
    out_img (np.array): The output image containing the fitted windows in green,
                        the binary points contributing to the left line regression in red,
                        the binary points contributing to the right line regression in blue and 
                        the rest of the binary points in white
    left_fit (np.array): Vector of coefficients for fitted second degree polynomial for left line
    right_fit (np.array): Vector of coefficients for fitted second degree polynomial for right line
    ploty (np.array): A numpy array with the range of x-pixels as values
    left_fitx (np.array): Fitted 2nd degree polynomial points for all "ploty-values" for left line
    right_fitx (np.array): Fitted 2nd degree polynomial points for all "ploty-values" for right line
    left and right line position of pixels being attributed to respective line:
        lefty, righty, leftx, rightx
    leftx_dir_marker (bool): Determines curve direction of left lane - if True -> right turn, 
                        if Fales -> left turn
    rightx_dir_marker (bool): Determines curve direction of right lane - if True -> right turn, 
                        if Fales -> left turn
    """
    # Take a histogram of the bottom half of the image
    histogram = np.sum(img[img.shape[0]/2:,:], axis=0)
    #plt.plot(histogram)
    
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((img, img, img))*255
    #plt.imshow(img)
    #plt.show()
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Choose the number of sliding windows
    nwindows = 8
    
    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Set the width of the windows +/- margin
    margin = 80
    # Set minimum number of pixels found to recenter window
    minpix = 60
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Create dictionary to collect window positions (already hast starting value in it)
    leftx_rep = {0:leftx_base}
    rightx_rep = {0:rightx_base}
    
    # Step through the windows one by one (first round) to identify correct window position
    # by taking previous image position as starting point and resentering the window position via
    # taking the mean of binaries in the initial window
    for window in range(nwindows):
        
        # Identify window boundaries in y direction
        win_y_low = img.shape[0] - (window+1) * window_height
        win_y_high = img.shape[0] - window * window_height
       
        # Identify window boundaries in x direction for both left and right and left lanes and 
        # left and right window side
        # Check whether current window position is available and use it (the case for first window 
        # as position was already defined in the dictionaries leftx_rep and rightx_rep)
        try:
            win_xleft_low = leftx_rep[window] - margin
            win_xleft_high = leftx_rep[window] + margin
            win_xright_low = rightx_rep[window] - margin
            win_xright_high = rightx_rep[window] + margin
        
        # Take the previous window as starting point (for all windows except the first one)
        except:
            win_xleft_low = leftx_rep[window-1] - margin
            win_xleft_high = leftx_rep[window-1] + margin
            win_xright_low = rightx_rep[window-1] - margin
            win_xright_high = rightx_rep[window-1] + margin
            

        
        # Identify the nonzero pixels in x and y within the currently identified window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # If > minpix pixels found, position the current left window on their mean 
        # position in x direction
        if len(good_left_inds) > minpix:
            leftx_rep[window] = np.int(np.mean(nonzerox[good_left_inds]))
        # Else position the current left window on the previous window's position in x direction
        else:
            if (window != 0):
                leftx_rep[window] = leftx_rep[window-1]
        # If > minpix pixels found, position the current right window on their mean position 
        # in x direction    
        if len(good_right_inds) > minpix:        
            rightx_rep[window] = np.int(np.mean(nonzerox[good_right_inds]))
        # Else position the current right window on the previous window's position in x direction
        else:
            if (window != 0):
               rightx_rep[window] = rightx_rep[window-1]
            
    # Reduce the mean of all window positions for left and right lane
    # from the mean of the starting window to determine the general direction of the turn
    # if negative -> right turn, if positive left turn
    
    leftx_dir = leftx_rep[0] - np.mean(list(leftx_rep.values()))
    rightx_dir = rightx_rep[0] - np.mean(list(rightx_rep.values()))
    
    # Create direction marker with right turn = True and left turn = False for both left and right line
    if leftx_dir < 0:
        leftx_dir_marker = True
    else:
        leftx_dir_marker = False
    
    if rightx_dir < 0:
        rightx_dir_marker = True
    else:
        rightx_dir_marker = False
    """
    # Not required anymore as lanes with different directions are taken care of in the final pipeline
    # If the direction markers for left and right line are pointing in different directions,
    # ensure that the direction of the line with a stronger turn will be used for both left and right 
    # direction marker
    if (leftx_dir_marker != rightx_dir_marker):
        if abs(leftx_dir) >= abs(rightx_dir):
            rightx_dir_marker = leftx_dir_marker
            
        else:
            leftx_dir_marker = rightx_dir_marker
    """

    
    # Step through the windows one by one (second round) to harmonize windows which seem to be biased
    # as they do not conform with general turn structure
    for window in range(nwindows):
        # Take all windows after the first one
        if window != 0:
            # Perform window adjustments for right turns for left and right line
            if rightx_dir_marker:
                # If the window position is left of the previous one (probably biased by some 
                # distortion on the left)
                # Right line
                if rightx_rep[window] <= rightx_rep[window-1]:
                    # Put window in the middle between previous and next window in x direction
                    try:
                        rightx_rep[window] = int(np.mean([rightx_rep[window - 1],
                                                          rightx_rep[window + 1]]))
                    # For last window take the position of previous window in x direction
                    except:
                        rightx_rep[window] = rightx_rep[window - 1]
                # Left line
                if leftx_rep[window] <= leftx_rep[window-1]:
                    # Put window in the middle between previous and next window in x direction
                    try:
                        leftx_rep[window] = int(np.mean([leftx_rep[window - 1],leftx_rep[window + 1]]))
                    # For last window take the position of previous window in x direction
                    except:
                        leftx_rep[window] = leftx_rep[window - 1]
                        
            # Perform window adjustments for left turns
            else:
                # If the window position is right of the previous one (probably biased by some 
                # distortion on the left)
                # Right line
                if rightx_rep[window] >= rightx_rep[window-1]:
                    # Put window in the middle between previous and next window in x direction
                    try:
                        rightx_rep[window] = int(np.mean([rightx_rep[window - 1],
                                                          rightx_rep[window + 1]]))
                    # For last window take the position of previous window in x direction
                    except:
                        rightx_rep[window] = rightx_rep[window - 1]
                if leftx_rep[window] >= leftx_rep[window-1]:
                    # Put window in the middle between previous and next window in x direction
                    try:
                        leftx_rep[window] = int(np.mean([leftx_rep[window - 1],leftx_rep[window + 1]]))
                    # For last window take the position of previous window in x direction
                    except:
                        leftx_rep[window] = leftx_rep[window - 1]
                        
        # Identify window boundaries in y direction
        win_y_low = img.shape[0] - (window+1) * window_height
        win_y_high = img.shape[0] - window * window_height      
        
        
        # Identify window boundaries in x direction for both left and right and left lanes and 
        # left and right window side       
        win_xleft_low = leftx_rep[window] - margin
        win_xleft_high = leftx_rep[window] + margin
        win_xright_low = rightx_rep[window] - margin
        win_xright_high = rightx_rep[window] + margin
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 4) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 4)   
    
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)    
    
    # Generate x and y values for plotting
    ploty = np.array(range(0,img.shape[0]))
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    return out_img, left_fit, right_fit, ploty, left_fitx, right_fitx, lefty, righty, leftx, rightx, leftx_dir_marker, rightx_dir_marker

In [ ]:
# Search for lane lines around line found in previous picture
def identify_lane_line_cont(img, left_fit, right_fit):
    """
    Input:
    img (np.array): The warped input image
    left_fit (np.array): Vector of coefficients for fitted second degree polynomial for left line 
                            of previous image
    right_fit (np.array): Vector of coefficients for fitted second degree polynomial for right line 
                            of previous image
    Output:
    out_img (np.array): The output image containing the fitted windows in green,
                        the binary points contributing to the left line regression in red, 
                        the binary points contributing to the right line regression in blue and 
                        the rest of the binary points in white
    left_fit (np.array): Vector of coefficients for fitted second degree polynomial for left line
    right_fit (np.array): Vector of coefficients for fitted second degree polynomial for right line
    ploty (np.array): A numpy array with the range of x-pixels as values
    left_fitx (np.array): Fitted 2nd degree polynomial points for all "ploty-values" for left line 
                            for current image
    right_fitx (np.array): Fitted 2nd degree polynomial points for all "ploty-values" for right line 
                            for current image
    leftx_dir_marker (bool): Determines curve direction of left lane - if True -> right turn, 
                        if Fales -> left turn
    rightx_dir_marker (bool): Determines curve direction of right lane - if True -> right turn, 
                        if Fales -> left turn
    """
    
    
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 80
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - 
                                   margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                                                           left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] -
                                    margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                                                            right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Determine the direction of the turns
    
    # For left line
    left_line_indicator = left_fitx[0]-left_fitx[len(ploty)-1]
    # Right turn
    leftx_dir_marker = True
    # Left turn
    if left_line_indicator < 0:
        leftx_dir_marker = False
        
    # For right lane
    right_line_indicator = right_fitx[0]-right_fitx[len(ploty)-1]
    # Right turn
    rightx_dir_marker = True
    # Left turn
    if right_line_indicator < 0:
        rightx_dir_marker = False
        
    
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((img, img, img))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    out_img = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    return out_img, left_fit, right_fit, ploty, left_fitx, right_fitx, lefty, righty, leftx, rightx, leftx_dir_marker, rightx_dir_marker

In [ ]:
# Function to plot and save an example image
def plot_identified_lane_image(out_img,ploty,left_fitx,right_fitx,ending):
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.axis("off")
    plt.savefig("output_images/{}.png".format(ending))
    plt.show()

In [ ]:
# Test of initial lane_line_identification (function identify_lane_line_first)
out_img, left_fit, right_fit, ploty, left_fitx, right_fitx, lefty, righty, leftx, rightx,_,_ = identify_lane_line_first(
    test_images_warped_cb["test_images/test2.jpg"])

plot_identified_lane_image(out_img,ploty,left_fitx,right_fitx,"19_identified_lanes_first")

# Test of search around line found in previous picture (function identify_lane_line_cont) 
# -> Test currently performed with same image as initial image
out_img, left_fit, right_fit, ploty, left_fitx, right_fitx, lefty, righty, leftx, rightx,_,_ = identify_lane_line_cont(
    test_images_warped_cb["test_images/test2.jpg"],left_fit, right_fit)

plot_identified_lane_image(out_img,ploty,left_fitx,right_fitx, "20_identified_lanes_second")

## 2.5 Calculate radius of curvature and position of vehicle

In [ ]:
def col_sob_comb(img_sobcomb,img_s_thresh):
    assert img_sobcomb.shape == img_s_thresh.shape, "not all input images have the same shape"
    combined_bin = np.zeros_like(img_sobcomb)
    combined_bin[(img_sobcomb == 1) | (img_s_thresh == 1)] = 1
    return combined_bin

In [ ]:
color_binary = {}
combined_binary = {}

for key in sorted(test_images.keys()):

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary[key] = np.dstack(( np.zeros_like(test_images_dir_grad[key]), test_images_sobcomb[key], 
                                   test_images_s_thresh[key]))

    # Combine the two binary thresholds
    combined_binary[key] = col_sob_comb(test_images_sobcomb[key],test_images_s_thresh[key])
    
plt.imsave("output_images/12_combined_binary.png",combined_binary["test_images/test2.jpg"],cmap = "gray")
    

for col,com in zip(sorted(color_binary.keys()),sorted(combined_binary.keys())):
    # Plotting thresholded images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.set_title('Stacked thresholds')
    ax1.imshow(color_binary[col])

    ax2.set_title('Combined S channel and gradient thresholds')
    ax2.imshow(combined_binary[com], cmap='gray')

### 2.2.4 Create overall function for binary creation

In [ ]:
# Function for creating thresholded binary image on single image (function summarizes all previous functions)
def img_to_thresh_bin(img,s_thresh_min=170, s_thresh_max=255, sobx_thresh_min=20, sobx_thresh_max=100, 
                      sobx_kernel_size=3, soby_thresh_min=20, soby_thresh_max = 100, soby_kernel_size = 3, 
                      mag_thresh_min = 20, mag_thresh_max = 100, mag_kernel_size = 3, dir_thresh_min = 0.4, 
                      dir_thresh_max = 0.8, dir_kernel_size = 15):

    # Create S-Channel binary pipeline
    hls = RGB_to_HLS(img)
    s = HLS_to_S(hls)
    s_thresh = S_to_thresh(s, s_thresh_min, s_thresh_max)
    
    # Create sobel binary pipeline
    sobelx = abs_sobel_thresh(img, orient='x', sobel_kernel= sobx_kernel_size, thresh=(sobx_thresh_min, sobx_thresh_max)) 
    sobely = abs_sobel_thresh(img, orient='y', sobel_kernel= soby_kernel_size, thresh=(soby_thresh_min, soby_thresh_max))
    mag_grad = mag_thresh(img, sobel_kernel= mag_kernel_size, mag_thresh=(mag_thresh_min, mag_thresh_max))
    dir_grad = dir_threshold(img, sobel_kernel=dir_kernel_size, thresh=(dir_thresh_min, dir_thresh_max))
    sob_comb = sob_to_sobcomb(sobelx,sobely,mag_grad,dir_grad)
    
    # Return overall combined binary
    return col_sob_comb(sob_comb,s_thresh)

## 2.3 Perform perspective transform